In [4]:
!pip install selenium
!pip install bs4

In [5]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.keys import Keys
import bs4
import time
import re

In [6]:
options = Options()
options.binary_location = r'C:\Program Files\Mozilla Firefox\firefox.exe'
driver = webdriver.Firefox(executable_path=r'C:\Users\mild1\.conda\envs\shopee\geckodriver.exe', options=options)
driver.get('https://shopee.co.th/')
time.sleep(5)
thai_button = driver.find_element_by_xpath('/html/body/div[2]/div[1]/div[1]/div/div[3]/div[1]/button')
thai_button.click()
time.sleep(3)
close_adver = driver.execute_script('return document.querySelector("shopee-banner-popup-stateful").shadowRoot.querySelector("div.shopee-popup__close-btn")')
close_adver.click()
time.sleep(3)
search = driver.find_element_by_xpath('/html/body/div[1]/div/header/div[2]/div/div[1]/div[1]/div/form/input')
search.send_keys('กาแฟผลไม้') #ใส่เนื้อหาที่ต้องการที่จะ Scrape จากเว็บ Shopee
search.send_keys(Keys.ENTER)

driver.execute_script("document.body.style.MozTransform='scale(0.1)';")
driver.execute_script('document.body.style.MozTransformOrigin = "0 0";')

records=[]
for i in range(3):
    data = driver.page_source
    soup = bs4.BeautifulSoup(data)
    All_product=soup.select(".row>div")
    for Product in All_product:
        name = Product.select_one("div.col-xs-2-4 > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1)").text.strip()
        try:
            p = Product.select_one("div.col-xs-2-4 > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(2) > div:nth-child(2) > span:nth-child(2)").text.strip()
            if p == '':
                print(1/0)
        except:
            p = Product.select_one("div.col-xs-2-4 > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(2) > div:nth-child(1)").text.strip()
        p = p.replace(",", "").replace("฿", "").replace(" ", "")
        try:
            start_price = float(p)
        except:
            price = p.rpartition('-')[0]
            start_price = float(price)

        sales = Product.select_one("div.col-xs-2-4 > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(3) > div:nth-child(2)").text.strip().replace("พัน","000").replace("ล้าน","000000")
        try:
            sales = float(re.sub('\D','',sales))
        except:
            sales = 0
        province = Product.select_one("div.col-xs-2-4 > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(4)").text.strip()
        #print(name, p)
        records.append([name, start_price,sales,province])
        
    time.sleep(5)
    next_button = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/div/div[2]/div[2]/div[3]/div/button[8]')
    next_button.click()
    time.sleep(5)
    
df = pd.DataFrame(records, columns=['Product','price','sales','province']) # ใส้ชื่อ Columns ทั้งหมด
df

,Product,price,sales,province
0,เมล็ดกาแฟคั่วดอยปางขอน อาราบิก้า 100% (5X200g.),544.0,61.0,จังหวัดเชียงราย
1,📌2 ถุง 260 ส่งฟรี📌เมล็ดกาแฟ อราบิก้าแท้ 100% ค...,260.0,16000.0,จังหวัดเชียงใหม่
2,📌300.-ได้ 2 ถุงส่งฟรี(คละได้)🔥เบลนใหม่ เมล็ดให...,300.0,3000.0,จังหวัดเชียงใหม่
3,#กาแฟยอดรัก #พร้อมส่ง ของแท้💯 2 ห่อ มี 60 ซอง ...,457.0,0.0,จังหวัดเชียงใหม่
4,เม็ดกาแฟ Brazil Santos บราซิล ซานโตส,150.0,0.0,จังหวัดกรุงเทพมหานคร
...,...,...,...,...
115,📌 คั่วกลางเข้ม - Full city roast 📌 กาแฟสด อราบ...,130.0,464.0,จังหวัดเชียงใหม่
116,PP Coffee กาแฟเพื่อสุขภาพ ที่มีสารสกัด 29ชนิด ...,350.0,0.0,จังหวัดกรุงเทพมหานคร
117,ส่งฟรี!!!! ( โปรโมชั่น ซื้อ 2 แถมฟรี 2 ) กาแฟ ...,459.0,162.0,จังหวัดลำพูน
118,เมล็ดกาแฟคั่วอ่อน BLEND 200g,250.0,5.0,จังหวัดเชียงราย
